# FECAF 1

In [1]:
import heapq
import pandas as pd;
import numpy as np
from stop_words import get_stop_words
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
df1 = pd.read_csv("Relatório sem título_Página 1_Tabela.csv")
df2 = pd.read_csv("Relatório sem título_Tabela.csv") 
# df1.head(3)
# df2.head(3)
# df_t.head(3)
# df3.head(3)
# df4.head(3)
# dfx.head(3)

,nome_blox,nome_curso,area_conhecimento,area_funcional,Record Count
0,Inovar para o sucesso!,Extensão em Negócios e Inovação,Inovação,Criatividade e Inovação,1874
1,O comportamento humano,Extensão em Negócios e Inovação,Relacionais,Gestão de Pessoas,1392
2,"Conceitos de linguagem e comunicação: fala, es...",Extensão em Negócios e Inovação,Relacionais,Marketing e Comunicação,1242


## limpesa e dummify do df1
### estamos trabalhando com 113 blox e 678 alunos

In [3]:
# df2["status"].value_counts()
# df1.loc[df1['nome_do_aluno'] == 'Manuel Marcos Vinicius Henry Melo']

In [4]:
df1.head()
dfx = df1
dfx = dfx.set_index('nome_do_aluno')
dfx = dfx.sort_values("nome_do_aluno")
entao = dfx['nome_blox']

In [5]:
def dummify(data, column_name):
    """
        Converte a coluna column_name em dummies / one-hot e as adiciona ao dataframe
        retorna uma copia do  df original *sem* a coluna que foi dummified
    """
    df = data.copy()
    # Nota: se você for um econometrista, pode se interessar por usar a opção dropFirst = True na get_dummies
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=None)], axis=1)
    return df2

def deletar(coluna):
    lista = list (coluna.unique()) #.tolinst()  
    for e in lista:
        coluna = coluna.replace(e,lista.index(e))
    return coluna

In [6]:
df1 = dummify(df1, ['nome_blox'])
lista = df1.columns.tolist()[5:]

In [7]:
aggregation_functions = {}
aggregation_functions['course_name'] = 'first'
for e in lista:
    aggregation_functions[e] = 'sum'
df1 = df1.groupby(df1['nome_do_aluno']).aggregate(aggregation_functions)

In [8]:
lista = ["course_name"]
for e in df1.columns[1:]:
    lista.append(e[10:])
df1.columns = lista
df1

,course_name,A Gestão de Pessoas Contemporânea,A avaliação educacional é uma tarefa didática,A experiência do consumidor 4.0,A formação de líderes,A interação homem-máquina,A magia da negociação,A sustentabilidade na era digital,Analisando dados estatísticos,Análise das Demonstrações Financeiras como foco no lucro,...,Tecnologias Digitais e a escola do futuro,Tecnologias Facilitadoras e exponenciais,Tipos e modelos de organizações,Todos conectados: redes de computadores,Tudo que você precisa saber sobre logística reversa,Um olhar para o futuro com as Tendências e os Cenários Econômicos,Uma caixa de ferramentas para o gerenciamento de projetos,Valorização profissional – Retenção de profissionais,Vamos entender o mercado! - Introdução ao Marketing,Ética e Responsabilidade Social
nome_do_aluno,,,,,,,,,,,,,,,,,,,,,
ABEL ALVES DE SOUZA,Extensão em Negócios e Inovação,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ADONES SALES,Pós-Graduação lato sensu em Processos Inovador...,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
ADRIANA CRISTINE VOIGT,Extensão em Negócios e Inovação,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
ADRIANO JOSÈ MIRANDA,Extensão em Negócios e Inovação,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
ADRIANO ZANELLA,Extensão em Negócios e Inovação,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
victor augusto gotordelli pereira,Extensão em Negócios e Inovação,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
wellington jose de miranda,Extensão em Negócios e Inovação,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
ÉVERLIN DE OLIVEIRA BARDDAL,Extensão em Negócios e Inovação,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Limpesa e tratamento do df2

## Primeira tentativa de unspervised learning do df1

In [9]:
df_t = pd.DataFrame()
for e in df1.columns[1:]:
    lista = list (df2.loc[df2['nome_blox'] == e ]["area_funcional"])
    df_t[e] = df1[e].replace(1, lista[0] )#.replace(0,np.nan)

In [10]:
# df_t.loc[df_t.index == 'Manuel Marcos Vinicius Henry Melo'].values
df_t.head()

,A Gestão de Pessoas Contemporânea,A avaliação educacional é uma tarefa didática,A experiência do consumidor 4.0,A formação de líderes,A interação homem-máquina,A magia da negociação,A sustentabilidade na era digital,Analisando dados estatísticos,Análise das Demonstrações Financeiras como foco no lucro,Aprender brincando: ludicidade e pensamento pedagógico,...,Tecnologias Digitais e a escola do futuro,Tecnologias Facilitadoras e exponenciais,Tipos e modelos de organizações,Todos conectados: redes de computadores,Tudo que você precisa saber sobre logística reversa,Um olhar para o futuro com as Tendências e os Cenários Econômicos,Uma caixa de ferramentas para o gerenciamento de projetos,Valorização profissional – Retenção de profissionais,Vamos entender o mercado! - Introdução ao Marketing,Ética e Responsabilidade Social
nome_do_aluno,,,,,,,,,,,,,,,,,,,,,
ABEL ALVES DE SOUZA,Gestão de Pessoas,0,0,0,0,0,0,0,0,Gestão Educacional,...,0,0,0,0,0,0,0,0,0,0
ADONES SALES,0,0,0,0,0,Vendas,0,0,0,0,...,0,0,0,0,Gestão Estratégica,0,0,Gestão de Pessoas,0,0
ADRIANA CRISTINE VOIGT,0,0,0,0,0,Vendas,0,Gestão e Administração,0,0,...,0,0,0,0,0,0,0,0,0,0
ADRIANO JOSÈ MIRANDA,0,0,0,0,0,Vendas,0,Gestão e Administração,0,0,...,0,0,0,0,0,0,0,Gestão de Pessoas,0,0
ADRIANO ZANELLA,0,0,0,0,0,Vendas,0,0,0,Gestão Educacional,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df3 = df1
df3 = df3.drop(list(df1.columns),axis=1)
lista = []
for e in df_t.values.tolist():
    lista.append(list(filter(lambda a: a != 0, e)))
df3['lista'] = lista
df3.head()

,lista
nome_do_aluno,
ABEL ALVES DE SOUZA,"[Gestão de Pessoas, Gestão Educacional, Gestão..."
ADONES SALES,"[Vendas, Gestão Estratégica, Contabilidade e T..."
ADRIANA CRISTINE VOIGT,"[Vendas, Gestão e Administração, Contabilidade..."
ADRIANO JOSÈ MIRANDA,"[Vendas, Gestão e Administração, Gestão Estrat..."
ADRIANO ZANELLA,"[Vendas, Gestão Educacional, Gestão Estratégic..."


In [12]:
# df3.loc[df3.index == "Manuel Marcos Vinicius Henry Melo" ]['lista'][0]

In [13]:
lll = []
for i in df3['lista']:
    k = i[0]
    lista = []
    for e in df2["area_funcional"].unique():
        lista.append(e.count(k))
    lll.append(lista)
df4 = pd.DataFrame.from_records(lll,columns = df2["area_funcional"].unique())
df4.head(3)

,Criatividade e Inovação,Gestão de Pessoas,Marketing e Comunicação,Empreendedorismo,Gestão Estratégica,Vendas,Contabilidade e Tributação,Gestão Educacional,Gestão e Administração,Legislação Educacional,Finanças,Database e Redes,Sustentabilidade,Produção e Processamento,Economia,Inovação em Educação,Gestão de Projetos,Direito e Administração
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
df4['corse_name'] = df1['course_name'].tolist()
df4['corse_name'] = df4["corse_name"].astype('category')

lista=[]
for e in dfx.index.unique():
    lista.append(dfx.loc[dfx.index == e]["nome_blox"].tolist())
df4["resp"] = lista
df4.head(2)

,Criatividade e Inovação,Gestão de Pessoas,Marketing e Comunicação,Empreendedorismo,Gestão Estratégica,Vendas,Contabilidade e Tributação,Gestão Educacional,Gestão e Administração,Legislação Educacional,Finanças,Database e Redes,Sustentabilidade,Produção e Processamento,Economia,Inovação em Educação,Gestão de Projetos,Direito e Administração,corse_name,resp
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Extensão em Negócios e Inovação,"[Cidadão consciente promove o desenvolvimento,..."
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,Pós-Graduação lato sensu em Processos Inovador...,[Planejar para empreender! Plano de negócios e...


In [15]:
df4.index = df3.index
# dfx = dfx.set_index('nome_do_aluno')
dfx = dfx.drop(["nome_blox","course_name","modalidade","ch","nota"],axis = 1)
dfx.cloumns = df4.columns
dfx.shape

C:\Users\Antonio Fonseca\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.


(5827, 0)

In [16]:
dfx = dfx.merge(df4, left_on= 'nome_do_aluno', right_on= 'nome_do_aluno')

In [17]:
# df3.loc[df3.index == "Manuel Marcos Vinicius Henry Melo" ]['lista'][0]

In [18]:
dfx = dummify(dfx, 'corse_name')
dfx['resp']= entao.astype('category')
dfx.head()

,Criatividade e Inovação,Gestão de Pessoas,Marketing e Comunicação,Empreendedorismo,Gestão Estratégica,Vendas,Contabilidade e Tributação,Gestão Educacional,Gestão e Administração,Legislação Educacional,...,Extensão em Negócios e Inovação,Pós-Graduação em Controladoria e Finanças,Pós-Graduação em Gerenciamento de Projetos,Pós-Graduação em Gestão Educacional e Novas Tecnologias,Pós-Graduação em Gestão Empresarial,Pós-Graduação em Gestão Estratégica de Pessoas,Pós-Graduação em Gestão da Inovação,Pós-Graduação em Liderança para Transformação Digital 4.0,Pós-Graduação lato sensu em Processos Inovadores de Ensino e Aprendizagem na Educação Básica (Sesi),Pós-Graduação lato sensu em Processos Inovadores de ensino e aprendizagem na Educação Profissional (Senai)
nome_do_aluno,,,,,,,,,,,,,,,,,,,,,
ABEL ALVES DE SOUZA,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
ABEL ALVES DE SOUZA,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
ABEL ALVES DE SOUZA,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
ABEL ALVES DE SOUZA,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
ABEL ALVES DE SOUZA,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [19]:
X_train, X_test, y_train, y_test = train_test_split((dfx.drop(["resp"],axis =1)) ,dfx["resp"], test_size=0.05,random_state=100)

In [20]:
len(y_train)

5535

In [21]:
# from sklearn.ensemble import RandomForestClassifier ,decision_function_shape='ovo'
from sklearn.svm import SVC
from sklearn.metrics import classification_report
def teste(X_train,y_train,X_test):
    rdm = SVC(kernel='linear', random_state=112)
    rdm = rdm.fit(X_train,y_train)


    y_pred = rdm.predict(X_test)
    j = rdm.decision_function(X_test)

    return y_pred, j

In [22]:
teste1 = teste(X_train,y_train,X_test)[0]
teste2 = teste(X_train,y_train,X_test)[1]
# teste2[0]

In [28]:
lis = y_train.unique()
def sugestao(X_train,y_train,X_test):
    rdm = SVC(kernel='linear', random_state=112)
    rdm = rdm.fit(X_train,y_train)
    y_pred = rdm.predict(X_test)
    j = list (rdm.decision_function(X_test))
    sugestoes = []
    t = 0
    for e in j:
        sugestao = []
        while len(sugestao)< 3:
            x = 3
            maiores_index = heapq.nlargest(x, range(len(e)), e.take)
            listinha = []
            for m in maiores_index:
                e = list(e)
                listinha.append(lis[m])
                sugestao.append(list(set(listinha) - set(df3["lista"][t])))
                
            x+=1
        sugestoes.append(sugestao[-1])
        t+=1
    return sugestoes

deus = sugestao(X_train,y_train,X_test)

In [24]:
df3["lista"][100]

['Vendas',
 'Gestão e Administração',
 'Gestão e Administração',
 'Contabilidade e Tributação',
 'Criatividade e Inovação',
 'Finanças',
 'Gestão Estratégica',
 'Gestão de Pessoas',
 'Gestão de Pessoas',
 'Empreendedorismo']

In [25]:
deus[100]

['O que você precisa saber sobre Linguagem SQL',
 'Competitividade de mercado',
 'Um olhar para o futuro com as Tendências e os Cenários Econômicos',
 'Gestão do conhecimento',
 'Tudo que você precisa saber sobre logística reversa']

In [26]:
#     for e in j:
#         unidade = []
#         while len(unidade) < 5:
#             x = 5
#             maiores_index = e[np.argpartition(e, x)]
#             for m in maiores_index:
#                 if lis(m) not in df3.loc[df3.index == "Manuel Marcos Vinicius Henry Melo" ]




#     e = j[0]
#     x = 5
#     listinha = []
#     maiores_index = heapq.nlargest(x, range(len(e)), e.take)
#     for m in maiores_index:
#         e = list(e)
#         p = e[m]
#         listinha.append(lis[m])